In [28]:
import torch
import torch.nn as nn

In [29]:
# Dictionary to set the magic constants
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers (# of transformer blocks)
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [30]:
# Layer normilization class
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim)) # make scale and shift trainable
        self.shift = nn.Parameter(torch.zeros(emb_dim)) # Start as 1 and 0, so they may learn to undo the z score step ?

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps) # eps helps avoid dividing by super small numbers (unstable)
        return self.scale * norm_x + self.shift 
        

In [31]:
# Feed forward class
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))
    
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            # Hidden layer neurons = 4 * len(emb dim)
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            # Non linear activation function
            GELU(),
            # Must squish output back down to emb dim so that each transformer block will output same length tensor
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [32]:
# Attention class
class MultiHeadAttention(nn.Module):
    #Constructor Inputs:d in is the embedding vector length, d out is the context vector length, context length is 
    # the number of tokens, dropout is the rate of data points left out, num_heads is the number of attention heads you want running
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )
# forward is called whenever class is called on a certain bacthed data (x is the batched data)
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method.

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec


In [33]:
#Creating a transformer block with all components
#two instances of layer norm bc they both have their own learned weights, only one instance of dropout in here bc it doesnt learn anything
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.att = MultiHeadAttention(
            d_in = cfg["emb_dim"], 
            d_out = cfg["emb_dim"], 
            context_length = cfg["context_length"], 
            dropout = cfg["drop_rate"], 
            num_heads = cfg["n_heads"]
            )
        self.dropout = nn.Dropout(cfg["drop_rate"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.feedforward = FeedForward(cfg)

    def forward(self, x):
        #Attention chunk of the transformer:

        #store original x as variable to add back later in shortcut step
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        s = self.dropout(x)
        x = x + shortcut

        #Feed forward chunk of the transformer:

        shortcut = x
        x = self.norm2(x)
        x = self.feedforward(x)
        x = self.dropout(x)
        s = x + shortcut

        return x



In [34]:
# Instantiate transformer block
block = TransformerBlock (GPT_CONFIG_124M)

In [35]:
input = torch.randn(3,4,768)
# 3 batches of 4 tokens of vector length 768
input

tensor([[[ 1.0508,  0.1115,  0.0418,  ..., -0.2717,  0.2040,  0.1343],
         [-1.5467, -1.7224,  0.6181,  ...,  0.3861,  0.7080,  0.3855],
         [-0.6041, -0.8325, -0.1790,  ..., -0.4612,  0.0048,  1.2171],
         [ 0.8821,  0.7194, -0.7811,  ...,  0.8666,  0.2268,  0.8948]],

        [[-1.0700,  1.5561, -0.3343,  ...,  1.0223, -0.6883,  2.2456],
         [ 1.3646, -0.6823, -1.0154,  ...,  0.5510,  1.9404, -0.9138],
         [-0.1407, -0.2456, -1.1382,  ...,  0.8361, -1.9936, -1.0801],
         [-0.4064, -0.4939, -0.2096,  ..., -3.3209, -0.9052, -1.2091]],

        [[ 0.5928, -0.1947,  0.9986,  ..., -1.4291, -0.2589, -0.3748],
         [ 1.5681,  0.5274,  0.3250,  ..., -0.3309, -0.4921, -1.5620],
         [ 0.1677,  0.1155,  0.6058,  ...,  0.0913,  1.1399, -1.0040],
         [ 0.3247, -0.4313, -0.2564,  ...,  0.4009,  0.3513, -0.3554]]])

In [36]:
# Is the output the same size as the input? Yay! 
output = block(input)
output.shape

torch.Size([3, 4, 768])